In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

# make sure that pandas datatype can be used in matplotlib plots without casting
register_matplotlib_converters()

# graphics customization
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['lines.linewidth'] = 1.5
darkgrey = '#3A3A3A'
lightgrey = '#414141'
barblue = plt.rcParams['axes.prop_cycle'].by_key()['color'][0]
plt.rcParams['text.color'] = darkgrey
plt.rcParams['axes.labelcolor'] = darkgrey
plt.rcParams['xtick.color'] = lightgrey
plt.rcParams['ytick.color'] = lightgrey



In [3]:
kanban_data = pd.read_csv("MK_data.csv",usecols=['Done', 'Type'], parse_dates=['Done']).dropna()
kanban_data.head(1)

,Done,Type
0,2023-09-26,Story


AttributeError: 'DataFrame' object has no attribute 'Done'